In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import datetime

def obtener_tiempo(dfr):
    #Esta funcion permite obtener una columna Datetime a partir del agno, dia, hora y minuto de los datos iniciales
    dfr['Day'] = pd.to_datetime(dfr['Day'], format='%j').dt.strftime('%m-%d')
    dfr['Hour'] = dfr['Hour'].astype(str).str.zfill(2)
    dfr['Datetime'] = pd.to_datetime(dfr[['Year','Day', 'Hour', 'Minute']]
                   .astype(str).apply(' '.join, 1), format='%Y %m-%d %H %M') 
    return dfr

def sustituir(dfr):
    #Sustitucion de los valores 9999... por NaN (sabemos que la fila 2 son todo valores nulos para cada variable)
    valores_null = dfr.iloc[2, 4:]
    for i in range(len(valores_null)):
        dfr.iloc[:,i+4] = dfr.iloc[:,i+4].replace(valores_null[i], np.nan)
    return dfr

def imputar_por_interpolacion(dfr):
    #Sustituye los valores NaN por valores aproximados mediante el metodo de interpolacion
    dfr = dfr.interpolate(method='linear', limit_direction='forward')
    return dfr

def imputar_por_KNND(dfr):
    #Este metodo es para imputar las filas que no se hayan podido imputar con interpolacion (primera fila)
    imputer = KNNImputer(n_neighbors=3, weights = 'distance')
    dfr[:] = imputer.fit_transform(dfr)
    return dfr

def normalizar_datos(dfr):
    #Dado un DataFrame, devuelve el DataFrame con valores normalizados
    x = dfr.values #returns a numpy array
    standard_scaler = preprocessing.StandardScaler()
    dfr[:] = standard_scaler.fit_transform(x)
    #dfr = pd.DataFrame(x_scaled)
    return dfr

def desnormalizar_datos(dfr):
    #Dado un DataFrame, devuelve el DataFrame con valores desnormalizados
    x = dfr.values
    standard_scaler = preprocessing.StandardScaler()
    dfr[:] = standard_scaler.inverse_transform(x)
    return dfr

def eliminar_gaps(dfr, n):
    #Eliminacion de filas donde alguna columna tiene N NaNs consecutivos
    for columna in range(dfr.shape[1]):
        mask = dfr.iloc[:,columna].notna()
        a = mask.ne(mask.shift()).cumsum()
        dfr = dfr[(a.groupby(a).transform('size') < n) | mask]
    return dfr

def NaN_consecutivos(dfr):
    #Obtencion del numero maximo de NaN consecutivos segun columna
    nans_consecutivos = []
    for columna in range(df.shape[1]):
        nan_columna = max(df.iloc[:,columna].isnull().astype(int).groupby(df.iloc[:,columna].notnull().astype(int).cumsum()).sum())
        nans_consecutivos.append(nan_columna)

    df_nans = pd.DataFrame(columns = ['Variable','Numero de nans consecutivos'])
    df_nans.iloc[:,0] = df.columns
    df_nans.iloc[:,1] = nans_consecutivos
    return df_nans

#1-Montamos Drive para poder acceder a los datos y los leemos

In [3]:
#Leemos los datos
#df = pd.read_csv('datos.csv', header=0)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv('/content/drive/MyDrive/TFG_codigo/Prediccion_general/datos.csv', header=0)

In [6]:
df.describe()

,Year,Day,Hour,Minute,IMF(nT),Bx GSM(nT),By GSM(nT),Bz GSM(nT),Flow Speed(km/s),Proton Density(n/cc),Proton Temperature(K)
count,2.749248e+06,2.749248e+06,2.749248e+06,2.749248e+06,2.749248e+06,2.749248e+06,2.749248e+06,2.749248e+06,2.749248e+06,2.749248e+06,2.749248e+06
mean,2.007570e+03,1.823232e+02,1.150000e+01,2.750000e+01,5.173788e+02,5.119720e+02,5.119870e+02,5.119146e+02,8.353063e+03,8.551741e+01,8.825655e+05
std,7.543063e+00,1.057835e+02,6.922188e+00,1.726027e+01,2.202735e+03,2.203992e+03,2.203989e+03,2.204005e+03,2.695064e+04,2.689619e+02,2.686965e+06
min,1.995000e+03,1.000000e+00,0.000000e+00,0.000000e+00,1.700000e-01,-4.857000e+01,-4.797000e+01,-5.537000e+01,2.312000e+02,3.000000e-02,1.564000e+03
25%,2.001000e+03,9.000000e+01,5.750000e+00,1.375000e+01,3.860000e+00,-2.470000e+00,-2.380000e+00,-1.690000e+00,3.569000e+02,3.400000e+00,3.523000e+04
50%,2.008000e+03,1.820000e+02,1.150000e+01,2.750000e+01,5.180000e+00,3.000000e-01,2.700000e-01,1.200000e-01,4.153000e+02,5.400000e+00,7.144900e+04
75%,2.014000e+03,2.740000e+02,1.725000e+01,4.125000e+01,7.220000e+00,3.000000e+00,2.990000e+00,2.080000e+00,5.153000e+02,9.280000e+00,1.453960e+05
max,2.021000e+03,3.660000e+02,2.300000e+01,5.500000e+01,9.999990e+03,9.999990e+03,9.999990e+03,9.999990e+03,9.999990e+04,9.999900e+02,9.999999e+06


In [10]:
df

,Year,Day,Hour,Minute,IMF(nT),Bx GSM(nT),By GSM(nT),Bz GSM(nT),Flow Speed(km/s),Proton Density(n/cc),Proton Temperature(K)
0,1995,1,0,0,1.37,0.130,1.17,-0.670,311.453358,18.422649,17401.558702
1,1995,1,0,5,1.26,0.090,1.12,-0.500,311.400000,18.460000,17347.000000
2,1995,1,0,10,1.46,0.037,1.30,-0.586,311.560000,18.348000,17510.600000
3,1995,1,0,15,1.66,-0.016,1.48,-0.672,311.720000,18.236000,17674.200000
4,1995,1,0,20,1.86,-0.069,1.66,-0.758,311.880000,18.124000,17837.800000
...,...,...,...,...,...,...,...,...,...,...,...
2749243,2021,49,23,35,2.87,2.130,0.03,-1.860,364.100000,4.670000,58514.000000
2749244,2021,49,23,40,2.92,2.210,0.07,-1.880,364.900000,4.400000,62883.000000
2749245,2021,49,23,45,2.90,2.450,0.05,-1.540,364.100000,4.650000,58624.000000
2749246,2021,49,23,50,2.92,2.380,0.13,-1.660,363.700000,4.810000,67353.000000


In [8]:
int(0.8 * int(df.shape[0]))

2199398

In [9]:
df.shape[0] - 2199398

549850

#2-Preprocesamos los datos y los normalizamos

In [7]:
df = sustituir(df)
df = imputar_por_interpolacion(df)
imputar_por_KNND(df.iloc[:3, :])
df = obtener_tiempo(df)
df = df.drop(['Year', 'Day', 'Hour', 'Minute'], axis = 1)
df = df.set_index('Datetime')
df = normalizar_datos(df)
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


KeyboardInterrupt: ignored

#3-Creacion de datasets

In [6]:
#===================================Parametros para obtener train_dataset y val_dataset====================================
TRAIN_SPLIT   = int(0.8 * int(df.shape[0]))
STEP          = 1
past_history  = 18
future_target = 4
BUFFER_SIZE   = 10000

#===================================PARAMETROS DE ENTRENAMIENTO====================================
LEARNING_RATE = 0.001       #Learning rate es la tasa de aprendizaje (mirar word para mas detalle)
EPOCHS        = 20          #Determina el numero de iteraciones que ser realizan sobre los datos de entrenamiento
PATIENCE      = 5
#EVALUATION_INTERVAL = 200
BATCH_SIZE    = 256
DROPOUT_LSTM  = 0.15

def multivariate_multioutput_data(dataset, target, start_index, end_index, history_size,
                      target_size, step, single_step=False):
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i-history_size, i, step)
        data.append(dataset[indices])

        if single_step:
            labels.append(target[i+target_size])
        else:
            labels.append(target[i:i+target_size])

    return np.array(data)[:,:,:,np.newaxis,np.newaxis], np.array(labels)[:,:,:,np.newaxis,np.newaxis]

#Creacion de datasets de entrenamiento y validacion
dataset = df.values

x_train_multi, y_train_multi = multivariate_multioutput_data(dataset, dataset, 0,
                                                 TRAIN_SPLIT, past_history,
                                                 future_target, STEP)
x_val_multi, y_val_multi = multivariate_multioutput_data(dataset, dataset,
                                             TRAIN_SPLIT, None, past_history,
                                             future_target, STEP)

train_data_multi = tf.data.Dataset.from_tensor_slices((x_train_multi, y_train_multi))
train_data_multi = train_data_multi.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_data_multi = tf.data.Dataset.from_tensor_slices((x_val_multi, y_val_multi))
val_data_multi = val_data_multi.batch(BATCH_SIZE).repeat()

#4-Creacion de modelos

**MODELO BASELINE**

In [9]:
def baseline(x_val, y_val):
  num_predicciones = len(y_val)
  timesteps_futuros = len(y_val[0])
  timesteps_pasados = len(x_val[0])
  variables = len(x_val[0][0])
  predicciones = np.zeros(shape=(y_val.shape))
  for prediccion in range(num_predicciones):
      dato = x_val[prediccion][timesteps_pasados-1]
      for num_timesteps_a_predecir in range(timesteps_futuros):
          predicciones[prediccion][num_timesteps_a_predecir] = dato
  return predicciones

**MODELO COMPLEJO**

In [10]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
def build_model_complejo(input_timesteps, output_timesteps, num_links, num_inputs):    
    model = Sequential()
    model.add(BatchNormalization(name = 'batch_norm_0', input_shape = (input_timesteps, num_inputs, 1, 1)))
    model.add(ConvLSTM2D(name ='conv_lstm_1',
                         filters = 64, kernel_size = (10, 1),                       
                         padding = 'same', 
                         return_sequences = True))
    
    model.add(Dropout(0.30, name = 'dropout_1'))
    model.add(BatchNormalization(name = 'batch_norm_1'))

    model.add(ConvLSTM2D(name ='conv_lstm_2',
                         filters = 64, kernel_size = (5, 1), 
                         padding='same',
                         return_sequences = False))
    
    model.add(Dropout(0.20, name = 'dropout_2'))
    model.add(BatchNormalization(name = 'batch_norm_2'))
    
    model.add(Flatten())
    model.add(RepeatVector(output_timesteps))
    model.add(Reshape((output_timesteps, num_inputs, 1, 64)))
    
    model.add(ConvLSTM2D(name ='conv_lstm_3',
                         filters = 64, kernel_size = (10, 1), 
                         padding='same',
                         return_sequences = True))
    
    model.add(Dropout(0.20, name = 'dropout_3'))
    model.add(BatchNormalization(name = 'batch_norm_3'))
    
    model.add(ConvLSTM2D(name ='conv_lstm_4',
                         filters = 64, kernel_size = (5, 1), 
                         padding='same',
                         return_sequences = True))
    
    model.add(TimeDistributed(Dense(units=1, name = 'dense_1', activation = 'relu')))
    model.add(Dense(units=1, name = 'dense_2', activation = 'linear'))
    
    optimizer = tf.keras.optimizers.RMSprop(lr=0.004, clipvalue=1.0)
    model.compile(loss = "mse", optimizer = optimizer, metrics = ['mae', 'mse'])
    return model

#5-Entrenamineto del modelo

In [9]:
import time
from tensorflow.keras.callbacks import CSVLogger, EarlyStopping
EPOCHS = 150
steps_per_epoch = 350
validation_steps = 500

path_checkpoint = "/content/drive/MyDrive/TFG_codigo/Prediccion_general/Modelos_2horasPasado/modelo_complejo_weights.h5"
modelckpt_callback = keras.callbacks.ModelCheckpoint(
    monitor="val_loss",
    filepath=path_checkpoint,
    verbose=1,
    save_weights_only=True,
    save_best_only=True,
)


modelstart = time.time()
early_stopping = EarlyStopping(monitor='val_loss', patience = 50, restore_best_weights=True)
model = build_model_complejo(x_train_multi.shape[1], future_target, y_train_multi.shape[2], x_train_multi.shape[2])
print(model.summary())

# Train
print("\nTRAIN MODEL...")
history = model.fit(train_data_multi,
                    epochs = EPOCHS,
                    validation_data=val_data_multi,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps,
                    verbose=1,
                    callbacks=[early_stopping, modelckpt_callback])

model.save('/content/drive/MyDrive/TFG_codigo/Prediccion_general/Modelos_2horasPasado/modelo_complejo.h5')
import pickle
with open('/content/drive/MyDrive/TFG_codigo/Prediccion_general/Modelos_2horasPasado/modelo_complejo_history', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)
print("\nModel Runtime: %0.2f Minutes"%((time.time() - modelstart)/60))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_norm_0 (BatchNormaliza (None, 18, 7, 1, 1)       4         
_________________________________________________________________
conv_lstm_1 (ConvLSTM2D)     (None, 18, 7, 1, 64)      166656    
_________________________________________________________________
dropout_1 (Dropout)          (None, 18, 7, 1, 64)      0         
_________________________________________________________________
batch_norm_1 (BatchNormaliza (None, 18, 7, 1, 64)      256       
_________________________________________________________________
conv_lstm_2 (ConvLSTM2D)     (None, 7, 1, 64)          164096    
_________________________________________________________________
dropout_2 (Dropout)          (None, 7, 1, 64)          0         
_________________________________________________________________
batch_norm_2 (BatchNormaliza (None, 7, 1, 64)          2

#6-Evaluación del modelo

**Realizamos predicciones con el modelo entrenado y con el modelo baseline, las guardamos y las cargamos**

In [12]:
predicciones = model.predict(x_val_multi, verbose=1)
predicciones_baseline = baseline(x_val_multi, y_val_multi)

np.save('/content/drive/MyDrive/TFG_codigo/Prediccion_general/Modelos_2horasPasado/modelo_complejo_predicciones', predicciones)
np.save('/content/drive/MyDrive/TFG_codigo/Prediccion_general/Modelos_2horasPasado/predicciones_baseline', predicciones_baseline)




17183/17183 [==============================] - 560s 33ms/step


In [11]:

predicciones_c        = np.load('/content/drive/MyDrive/TFG_codigo/Prediccion_general/Modelos_2horasPasado/modelo_complejo_predicciones.npy')
predicciones_baseline = np.load('/content/drive/MyDrive/TFG_codigo/Prediccion_general/Modelos_2horasPasado/predicciones_baseline.npy')

Generamos la estructura para que con las funciones podamos obtener las métricas de cada modelo para compararlos

In [12]:
predicciones = [predicciones_c, predicciones_baseline]
nombre_modelos = ['Complejo', 'Baseline']
variables = df.columns

In [16]:
variables = variables[4:]

In [17]:
variables

Index(['IMF(nT)', 'Bx GSM(nT)', 'By GSM(nT)', 'Bz GSM(nT)', 'Flow Speed(km/s)',
       'Proton Density(n/cc)', 'Proton Temperature(K)'],
      dtype='object')

**Generamos las comparativas entre los modelos**

In [13]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
def obtener_metricas(y_val, y_preds, nombre_modelos):
    #Aplanamos los datos para que se puedan calcular las metricas
    y_val = np.squeeze(y_val)
    y_val = y_val.ravel()
    datos_mse  = []
    datos_r2   = []
    datos_rmse = []
    datos_mae  = []
    columnas   = []
    for modelo in range(len(nombre_modelos)):
        y_pred = np.squeeze(y_preds[modelo])
        y_pred = y_pred.ravel()
        #Calculamos las metricas
        r2   = r2_score(y_val, y_pred)
        mse  = mean_squared_error(y_val, y_pred)
        mae  = mean_absolute_error(y_val, y_pred)
        rmse = mse**0.5
        #Los anadimos a las listas correspondientes
        datos_mse.append(mse)
        datos_r2.append(r2)
        datos_rmse.append(rmse)
        datos_mae.append(mae)
        #Creamos una columna por modelo
        columnas.append(nombre_modelos[modelo])

    #Mostramos los resultados en forma de df
    nombres_metricas = ['R2', 'RMSE', 'MSE', 'MAE']
    valores = [datos_r2, datos_rmse, datos_mse, datos_mae]

    metricas = pd.DataFrame(valores, columns = columnas, index = nombres_metricas)
    return metricas



def desglose_por_timestep(y_val, predicciones):
    num_predicciones = predicciones.shape[0]
    num_timesteps   = predicciones.shape[1]

    timesteps_y_val  = []
    timesteps_y_pred = []
    
    #Creamos 4 listas, una para cada timestep
    for lista_timestep in range(num_timesteps):
        timestep_valN = []
        timesteps_y_val.append(timestep_valN)
        timestep_predN = []
        timesteps_y_pred.append(timestep_predN)
    #Metemos en cada una de los 4 listas, el correspondiente timestep de cada prediccion
    for prediccion in range(num_predicciones):
        for timestep in range(num_timesteps):
            timesteps_y_pred[timestep].append(predicciones[prediccion][timestep])
            timesteps_y_val[timestep].append(y_val[prediccion][timestep])
    
    return timesteps_y_val, timesteps_y_pred

def comparacion_modelos_timestep(y_val, predicciones, nombre_modelos):
  timesteps_y_preds = []
  for modelo in range(len(nombre_modelos)):
      timesteps_y_val, timesteps_y_pred_modelo = desglose_por_timestep(y_val, predicciones[modelo])
      timesteps_y_preds.append(timesteps_y_pred_modelo)
  for timestep in range(4):
      timesteps_modelos = []
      for modelo in range(len(nombre_modelos)):
          timesteps_modelos.append(timesteps_y_preds[modelo][timestep])
      print("=========TIMESTEP: ",timestep,"=========\n",obtener_metricas(timesteps_y_val[timestep], timesteps_modelos, nombre_modelos))


def comparacion_modelos_timestep_variables(y_val, predicciones, nombre_modelos, nombre_variables):
  desglose_preds = []
  for modelo in range(len(nombre_modelos)):
      desglose_val, desglose_pred = agrupar_variables_timestep(y_val, predicciones[modelo])
      desglose_preds.append(desglose_pred)
  
  for paso in range(4):
    print("===================TIMESTEP ", paso,"===================")
    for variable in range(len(nombre_variables)):
        variableN_modelos = []
        for modelo in range(len(nombre_modelos)):
          variableN_modelos.append(desglose_preds[modelo][paso][variable])
        print("=====VARIABLE", nombre_variables[variable],"====")
        print(obtener_metricas(desglose_val[paso][variable], variableN_modelos, nombre_modelos))


def agrupar_variables_timestep(y_val, y_pred):
    num_predicciones = y_val.shape[0]
    num_timesteps    = y_val.shape[1]
    num_variables    = y_val.shape[2]
    #Creacion de listas
    desglose_val  = []
    desglose_pred = []
    for paso in range(num_timesteps):
        timestepN_val  = []
        timestepN_pred = []
        for i in range(num_variables):
            timestepN_varI_val = []
            timestepN_val.append(timestepN_varI_val)
            
            timestepN_varI_pred = []
            timestepN_pred.append(timestepN_varI_pred)
            
        desglose_val.append(timestepN_val)
        desglose_pred.append(timestepN_pred)
    
    #Desglose
    for prediccion in range(num_predicciones):
        for paso in range(num_timesteps):
            for var in range(num_variables):
                desglose_val[paso][var].append(y_val[prediccion][paso][var])
                desglose_pred[paso][var].append(y_pred[prediccion][paso][var])
    
    return desglose_val, desglose_pred




def resumen_comparativa(y_val, predicciones, nombre_modelos, variables):
    pd.set_option("display.max_rows", None, "display.max_columns", None)

    #1-Comparativa general de los modelos
    print("===================================================COMPARATIVA GENERAL DE LOS MODELOS===================================================")
    print(obtener_metricas(y_val, predicciones, nombre_modelos))

    #2-Comparativa por timestep de los modelos
    print("===================================================COMPARATIVA POR TIMESTEP DE LOS MODELOS===================================================")
    print(comparacion_modelos_timestep(y_val, predicciones, nombre_modelos))

    print("===================================================COMPARATIVA POR TIMESTEP Y VARIABLES DE LOS MODELOS===================================================")
    #3-Comparativa por timestep y variable de los modelos
    print(comparacion_modelos_timestep_variables(y_val, predicciones, nombre_modelos, variables))

In [18]:
resumen_comparativa(y_val_multi, predicciones, nombre_modelos, variables)

===================================================COMPARATIVA GENERAL DE LOS MODELOS===================================================
      Complejo  Baseline
R2    0.888201  0.881893
RMSE  0.292120  0.300248
MSE   0.085334  0.090149
MAE   0.161063  0.150333
===================================================COMPARATIVA POR TIMESTEP DE LOS MODELOS===================================================
=========TIMESTEP:  0 =========
       Complejo  Baseline
R2    0.943110  0.948454
RMSE  0.208384  0.198356
MSE   0.043424  0.039345
MAE   0.115311  0.097533
=========TIMESTEP:  1 =========
       Complejo  Baseline
R2    0.899649  0.897183
RMSE  0.276761  0.280141
MSE   0.076596  0.078479
MAE   0.153703  0.141391
=========TIMESTEP:  2 =========
       Complejo  Baseline
R2    0.867839  0.857743
RMSE  0.317608  0.329517
MSE   0.100875  0.108582
MAE   0.178455  0.170097
=========TIMESTEP:  3 =========
       Complejo  Baseline
R2    0.842205  0.824191
RMSE  0.347044  0.366319
MSE   0.120440